In [1]:
from PIL import Image
import requests, sqlite3,tqdm
from io import BytesIO
import pandas as pd

%load_ext autoreload
%autoreload 2
import dashDBHelpers as hlprs

dbName = "sqliteDb/confMatrixDB.db"

hlprs.createMatrixTables(dbName)
hlprs.createItemTable(dbName)
hlprs.insertItemData(dbName)

Index(['_id', 'baseParentId', 'baseParentType', 'created', 'creatorId',
       'description', 'folderId', 'lowerName', 'name', 'size', 'updated',
       'largeImage.fileId', 'largeImage.sourceName',
       'meta.npSchema.controlSlide', 'meta.npSchema.stainID',
       'meta.npSchema.blockID', 'meta.npSchema.caseID',
       'meta.npSchema.regionName', 'meta.npWorking.stainID', 'meta.artifacts',
       'meta.bad_scan', 'meta.control', 'meta.jc_confirm', 'meta.notes',
       'meta.region', 'meta.stain', 'meta.subject', 'meta.ABC',
       'meta.Age Death', 'meta.Age Onset', 'meta.ApoE', 'meta.Braak Stage',
       'meta.CERAD', 'meta.Cause of Death', 'meta.Clinical Dx',
       'meta.Comments', 'meta.Duration', 'meta.Family History',
       'meta.Other NP Dx - AD', 'meta.Other NP Dx - LBD',
       'meta.Other NP Dx - Misc 1', 'meta.Other NP Dx - Misc 2',
       'meta.Other NP Dx - Misc 3', 'meta.Other NP Dx - Misc 4',
       'meta.Other NP Dx - TAU', 'meta.Other NP Dx - TDP',
       'meta.Oth

In [2]:
from transformers import pipeline
classifer = pipeline("image-classification",model="models/stainID_model/")

/home/dagutman/.local/lib/python3.8/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [3]:
ds = hlprs.queryTable(dbName,"dsai") ## Get the list of items in the DSA, in future will merge this with model responses
df = pd.DataFrame(ds)


In [4]:
dsaBaseUrl =  "https://styx.neurology.emory.edu/girder/api/v1"
from PIL import Image
import requests
from io import BytesIO
dsaBaseUrl = "https://styx.neurology.emory.edu/girder/api/v1"



In [5]:
import tqdm
con = sqlite3.connect(dbName)
c = con.cursor()


for row_dict in tqdm.tqdm(df.to_dict(orient="records")):
    #print(row_dict)
    imgId = row_dict['_id']
    thumbUrl = f'{dsaBaseUrl}/item/{imgId}/tiles/thumbnail'
    response = requests.get(thumbUrl)
    img = Image.open(BytesIO(response.content))
#     img.show()
    
    modelOutput = classifer.predict(img)
    dataValues = ('stainPredV1',row_dict['_id'], modelOutput[0]['label'], str(modelOutput))

    sql = 'insert into modelResponses ( modelName, imageId , modelLabel, modelResponse ) values (?,?,?,?)'
    c.execute(sql,dataValues)
    con.commit()
con.close()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1231/1231 [05:59<00:00,  3.42it/s]


[(1, 'stainPredV1', '6105948c68f4fe34fca1bf7c', 'Abeta', "[{'score': 0.9900652766227722, 'label': 'Abeta'}, {'score': 0.0012225569225847721, 'label': 'He'}, {'score': 0.0011039278469979763, 'label': 'Ptdp'}, {'score': 0.0010679649421945214, 'label': 'Syn'}, {'score': 0.0010232857894152403, 'label': 'Lfb'}]"), (2, 'stainPredV1', '6105948c68f4fe34fca1bf7f', 'Abeta', "[{'score': 0.9900327920913696, 'label': 'Abeta'}, {'score': 0.0012436920078471303, 'label': 'He'}, {'score': 0.0010970975272357464, 'label': 'Ptdp'}, {'score': 0.0010546785779297352, 'label': 'Syn'}, {'score': 0.0010363818146288395, 'label': 'Lfb'}]"), (3, 'stainPredV1', '6105948c68f4fe34fca1bf82', 'Biels', "[{'score': 0.9879933595657349, 'label': 'Biels'}, {'score': 0.0019348063506186008, 'label': 'Lfb'}, {'score': 0.001496260752901435, 'label': 'He'}, {'score': 0.0012056923005729914, 'label': 'Cp13'}, {'score': 0.0011832965537905693, 'label': 'Ubiq'}]"), (4, 'stainPredV1', '6105948d68f4fe34fca1bf85', 'Biels', "[{'score': 0

In [ ]:
dsaBaseUrl = "https://styx.neurology.emory.edu/girder/api/v1"
predictionData = []


#for i in tqdm.tqdm(imageSetList[0:5]):
for i in tqdm.tqdm(imageSetList):
    if 'npSchema' in i['meta'] and 'stainID' in i['meta']['npSchema']:
        npStain = i['meta']['npSchema']['stainID']

        
        thumbUrl = f'{dsaBaseUrl}/item/{i["_id"]}/tiles/thumbnail'
        response = requests.get(thumbUrl)

        c = classifer.predict(Image.open(BytesIO(response.content)))
#         print(npStain,i['name'],c[0]['label'])
        
        p = {'currentStain':npStain,'imageName':i['name'],'predictedStain':c[0]['label'],'predictedConf':c[0]['score'],'allPredictions': c}
        predictionData.append(p)
#         print(c)


In [ ]:
with open("predictionSample.json","w") as fp:
    json.dump(predictionData,fp)

In [ ]:
from sklearn.metrics import confusion_matrix
import pylab as pl
y_test=['business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business']

# pred=array(['health', 'business', 'business', 'business', 'business',
#        'business', 'health', 'health', 'business', 'business', 'business',
#        'business', 'business', 'business', 'business', 'business',
#        'health', 'health', 'business', 'health'], 
#       dtype='|S8')

cm = confusion_matrix(y_test, pred)
pl.matshow(cm)
pl.title('Confusion matrix of the classifier')
pl.colorbar()
pl.show()

In [ ]:
y_test = [y['currentStain'] for y in predictionData]
y_pred = [y['predictedStain'] for y in predictionData]

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)

In [ ]:

# display_labels =['aBeta','Biels','HE','Tau','pTDP']

# cm = metrics.confusion_matrix(y_test,y_pred)
# cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = cm, display_labels=display_labels)
# cm_display.plot()
# plt.show()



In [ ]:

lbl = df['currentStain'].value_counts() > 100

df1 = df[df['currentStain'].map(df['currentStain'].value_counts()) > 100]
df1.currentStain.unique()


In [ ]:
lblSet = ['HE','Tau','Syn','aBeta','pTDP','Biels']
from sklearn import metrics
cm = metrics.confusion_matrix( list(df1.currentStain.values), list(df1.predictedStain.values), labels=df1.currentStain.unique())

fig = metrics.ConfusionMatrixDisplay(cm,display_labels=df1.currentStain.unique())
fig.plot()

In [ ]:
px.scatter(cm,x='')

In [ ]:
cm
# import plotly.express as px

# px.histogram(df1,x='currentStain')

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd
# from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
# from sklearn.preprocessing import LabelEncoder

# ex_classes = {
#     "Classes": [
#         "bicep_curls",
#         "dumbbell_rows",
#         "dumbbell_shoulder_press",
#         "jumping_jacks",
#         "lateral_shoulder_raises",
#         "lunges",
#         "pushups",
#         "situps",
#         "squats",
#         "tricep_extensions",
#     ]
# }
# df_classes = pd.DataFrame(data=ex_classes)
# label_enc = LabelEncoder()
# label_enc.fit(df_classes["Classes"])

# y_test = np.asarray(
#     [8, 8, 8, 6, 6, 6, 2, 2, 2, 5, 5, 5, 1, 1, 1, 7, 7, 7, 9, 9, 9, 0, 0, 0, 0, 4]
# )
# y_pred = np.asarray(
#     [8, 4, 4, 6, 6, 6, 2, 2, 2, 5, 5, 5, 1, 1, 1, 9, 7, 7, 9, 9, 9, 0, 0, 1, 0, 4]
# )
# cm = confusion_matrix(y_test, y_pred, labels=np.arange(len(df_classes)))
# display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_enc.classes_)
# fig, ax = plt.subplots(figsize=(10, 10))
# display.plot(ax=ax, xticks_rotation="vertical")
# plt.show()

In [ ]:
import numpy as num
import matplotlib.pyplot as plot

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay


iris = datasets.load_iris()
x = iris.data
y = iris.target
class_names = iris.target_names


x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)


classifier = svm.SVC(kernel="linear", C=0.02).fit(x_train, y_train)

num.set_printoptions(precision=2)

title_options = [
    ("Confusion matrix, without normalization", None),
    ("Normalized confusion matrix", "true"),
]
for title, normalize in title_options:
    display = ConfusionMatrixDisplay.from_estimator(
        classifier,
        x_test,
        y_test,
        display_labels=class_names,
        cmap=plot.cm.Blues,
        normalize=normalize,
    )
    display.ax_.set_title(title)

    print(title)
    print(display.confusion_matrix)

plot.show()

In [ ]:
import pandas as pd
from sklearn import metrics
import json
with open("predictionSample.json", "r") as fp:
    preds = json.load(fp)

df_pred = pd.DataFrame(preds)

lblSet = ['HE', 'Tau', 'Syn', 'aBeta', 'pTDP', 'Biels']
df1 = df_pred[df_pred['currentStain'].map(
    df_pred['currentStain'].value_counts()) > 100]

df1 = df1.replace({"Abeta": "aBeta", "Ptdp": "pTDP", "He": "HE"})
df1.drop(columns=['allPredictions'])
cm = metrics.confusion_matrix(list(df1.currentStain.values), list(
    df1.predictedStain.values), labels=df1.currentStain.unique())

In [ ]:
df1[df1.currentStainntStain=='Biels']